In [35]:
 # THE GOAL OF THIS CODE IS TO TAKE THE CODE ALREADY MADE IN HEURISTIC ADVANCED (SPLITS DATA INTO DESTINATIONS, ASSIGNS 
    # PALLETS TO PLANES, THEN ASSIGNS PALLETS TO PALLET POSITIONS) AND MAKE IT SO THE CODE DOES NOT HAVE ANYTHING HARD CODED
    
# PACKAGES
from __future__ import print_function
#from ortools.sat.python import cp_model
import os
import pandas as pd
import pyomo
import numpy
import pprint
from numpy import genfromtxt
from pandas import ExcelWriter
from pandas import ExcelFile
import math

# change working directory
os.chdir("C:/Users/Jacob Lindell/Desktop/All/Work/ATAK/Optimization/ACLPP/Subproblems/1 B 1P w S/Sequential")

In [36]:
# DATA
Pallet_Data = pd.read_csv('Pallet_Data.csv', header=None)
#Pallet_Data = pd.read_csv('Pallet_Data_ext.csv', header=None)
# col 1 = priority, col 2 = weight, col 3 = dest (1 = CO, 2 = Scott, 3 = McGuire), col 4 = hold time

Schedule = pd.read_csv('Schedule.csv', header=None)
#Schedule_bin = pd.read_csv('Schedule_bin.csv', header=None)

In [37]:
 # SPLIT DATA BY DESTINATION

Pallet_Data[4] = numpy.arange(len(Pallet_Data))
# pallet #, dest, priority, hold time, weight
Pallet_Data = Pallet_Data[[4,2,0,3,1]]
Pallet_Data = Pallet_Data.T.reset_index(drop=True).T
Pallet_Data = Pallet_Data.reset_index(drop=True)
# pre sort the pallet data for ease of use later
Pallet_Data = Pallet_Data.sort_values(by=[1,2,3], ascending = [True, True, False])

# this is where the pallet data will get going for each destination
## Destsplit is (3,102,4)
Destsplit = numpy.zeros(shape=(Schedule[0].max(), len(Pallet_Data), len(Pallet_Data.columns)-1), dtype = int)

# this will be the counter for all of the destinations
counts = numpy.ones(Schedule[0].max(), dtype=int)

# Loop to split data into datasets for each destination. 
for i in range(0, len(Pallet_Data)):
    Dest = Pallet_Data.iloc[i,1]
    Dest = Dest - 1
    c = counts[Dest]
    Destsplit[Dest][c][0] = Pallet_Data.iloc[i,0]
    Destsplit[Dest][c][1] = Pallet_Data.iloc[i,2]
    Destsplit[Dest][c][2] = Pallet_Data.iloc[i,3]
    Destsplit[Dest][c][3] = Pallet_Data.iloc[i,4]
    counts[Dest] = counts[Dest] + 1

In [38]:
# ASSIGN PALLETS TO PLANES

# Destflight is going to be split up as 3x3x34x4 (destination x flights x pallet positions x columns)
# Destsplit is going to be split up by 3x102x4 (destination x pallets x columns)
Max_PP = 34
dest = Schedule[1].max()
flights = Schedule.iloc[:,1].max()
priority = Pallet_Data[2].max()
pallets = len(Pallet_Data)
# columns = #, priority, hold time, weight
Destflight = numpy.zeros(shape=(dest, flights, Max_PP, len(Pallet_Data.columns)-1), dtype=int)



Max_wt = 120000

Wt = numpy.zeros(shape=(flights,dest), dtype=int)
PPused = numpy.zeros(shape=(flights,dest), dtype=int)
Pallet_ass = []
Pallet_ass.append(0)
counts = numpy.ones(shape=(flights,dest), dtype=int)


for d in range(0,dest):
    for f in range(0,flights):
        for p in range(1,priority+1):
            for u in range(0,pallets):
                if Destsplit[d][u][0] in Pallet_ass:
                    pass
                else:
                    if Wt[f][d] + Destsplit[d][u][3] < Max_wt and PPused[f][d] < Max_PP:
                        if Destsplit[d][u][1] == p:
                            Destflight[d][f][counts[d][f]][0] = Destsplit[d][u][0]
                            Destflight[d][f][counts[d][f]][1] = Destsplit[d][u][1]
                            Destflight[d][f][counts[d][f]][2] = Destsplit[d][u][2]
                            Destflight[d][f][counts[d][f]][3] = Destsplit[d][u][3]
                            
                            Wt[f][d] = Wt[f][d] + Destsplit[d][u][3]
                            Pallet_ass.append(Destsplit[d][u][0])
                            PPused[f][d] = PPused[f][d] + 1
                            counts[f][d] = counts[f][d] + 1
                        else:
                            pass
                    else:
                        pass

In [53]:
# ASSIGN PALLETS TO PALLET POSITIONS

# From Ericka's paper
Dy = [68, 60, 52, 44, 36, 28, 20, 12, 4, -4, -12, -20, -28, -36, -44, -52, -60, -68]
Dx = [-4.75, 4.75]

# Allowable pallet positions based on x and y coordinates
PPlength = 18 
PPwidth = 2
CoG = 0
CoG_min = -100
CoG_max = 50
CoG_new = 0

FlightPP = numpy.zeros(shape=(dest, flights, PPlength, PPwidth), dtype=int)

for d in range(0,dest):
    for f in range(0,flights):
        Destflight[d][f] = sorted(Destflight[d][f], key=lambda x: x[3], reverse=True)
        for u in range(0,pallets):
            Fx = 0
            Fy = 0
            Fx_sum = 0
            Fy_sum = 0
            Wt_ass = 0
            if FlightPP[0][0].sum() == 0:
                        break
            else:
                for x in range(0,PPwidth):
                    for y in range(0,PPlength):
                        Fx_sum = Fx_sum + Dx[x] * FlightPP[d][f][y][x]
                        Fy_sum = Fy_sum + Dy[y] *FlightPP[d][f][y][x]
                        Wt_ass = Wt_ass + FlightPP[d][f][y][x]
                CoGy_new = Fy_sum/Wt_ass
                CoGx_new = Fx_sum/Wt_ass
            
            if CoG_min <= CoG_new <= CoG_max:
                if Fy_sum <= 0:
                    for l in range(0,PPlength):
                        if FlightPP[d][f][l][0] != 0:
                            if FlightPP[d][f][l][1] != 0:
                                FlightPP[d][f][l-1][0] = Destflight[d][f][u][0]
                                break
                            else:
                                if Fx_sum <= 0:
                                    FlightPP[d][f][l-1][1] = Destflight[d][f][u][0]
                                    break
                                else:
                                    FlightPP[d][f][l-1][0] = Destflight[d][f][u][0]
                                    break
                else:
                    for l in range(PPlength-1,0,-1):
                        if FlightPP[d][f][l][0] != 0:
                            if FlightPP[d][f][l+1][1] != 0:
                                FlightPP[d][f][l+1][0] = Destflight[d][f][u][0]
                                break
                            else:
                                if Fx_sum < 0:
                                    FlightPP[d][f][l+1][1] = Destflight[d][f][u][0]
                                    break
                                else:
                                    FlightPP[d][f][l+1][0] = Destflight[d][f][u][0]

In [54]:
FlightPP[0][0]

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [43]:
Destflight[0][0]

array([[   0,    0,    0,    0],
       [  15,    1,   89, 5191],
       [  38,    1,   89, 5708],
       [   4,    1,   78, 9970],
       [  29,    1,   60, 8707],
       [  45,    1,   57, 7444],
       [  92,    1,   50, 6734],
       [  72,    1,   45, 5834],
       [  42,    1,   33, 6746],
       [  99,    1,   17, 8164],
       [  88,    1,   16, 7300],
       [  24,    1,    8, 3483],
       [  39,    2,   99, 3449],
       [  12,    2,   60, 9335],
       [  11,    2,   52, 3750],
       [  52,    2,   46, 8260],
       [  49,    2,   45, 9390],
       [   6,    2,   42, 8571],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [  